In [1]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import re
import pandas as pd
import contractions

contractions.add("c'mon", 'come on')

file = pd.read_csv('friends_chat.txt',sep='\t')
lines = file['line']

print('Initial chats')
for i in range(10):
    print(lines[i])
    
def cleanchat(line):
    #to convert text to lower case
    line = line.lower()
    #to remove ending EOL
    line = re.sub(r'\n','',line) 
    #re-format punctuations
    line = re.sub(r"[-()]", "", line)
    line = re.sub(r"\.", " .", line)
    line = re.sub(r"\!", " !", line)
    line = re.sub(r"\?", " ?", line)
    line = re.sub(r"\,", " ,", line)
    
    #string replacement
    line = re.sub(r"i'm", "i am", line)
    line = re.sub(r"he's", "he is", line)
    line = re.sub(r"she's", "she is", line)
    line = re.sub(r"it's", "it is", line)
    line = re.sub(r"that's", "that is", line)
    line = re.sub(r"what's", "that is", line)
    line = re.sub(r"\'ll", " will", line)
    line = re.sub(r"\'re", " are", line)
    line = re.sub(r"won't", "will not", line)
    line = re.sub(r"can't", "cannot", line)
    line = re.sub(r"n't", " not", line)
    line = re.sub(r"n'", "ng", line)
    line = re.sub(r"ohh", "oh", line)
    line = re.sub(r"ohhh", "oh", line)
    line = re.sub(r"ohhhh", "oh", line)
    line = re.sub(r"ohhhhh", "oh", line)
    line = re.sub(r"ohhhhhh", "oh", line)
    line = re.sub(r"ahh", "ah", line)
    
    return line
    

Initial chats
There's nothing to tell! He's just some guy I work with!
C'mon, you're going out with the guy! There's gotta be something wrong with him!
Alright Joey, be nice. So does he have a hump? A hump and a hairpiece?
Wait, does he eat chalk?
Just, 'cause, I don't want her to go through what I went through with Carl- oh!
Okay, everybody relax. This is not even a date. It's just two people going out to dinner and not having sex.
Sounds like a date to me.
Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
Oh, yeah. Had that dream.
Then I look down, and I realize there's a phone... there.


In [2]:
def preprocess(x):
    clean = cleanchat(x)
    processed = contractions.fix(clean)
    return processed

file['processed_text'] = file['line'].apply(preprocess)


In [3]:
def length(x):
    return len(x.split())
file['length'] = file['line'].apply(length)

In [4]:
file['length'].describe()

count    60849.000000
mean        10.161416
std         10.438841
min          1.000000
25%          3.000000
50%          7.000000
75%         14.000000
max        195.000000
Name: length, dtype: float64

In [5]:
MAX_WORD_LENGTH = 15

In [6]:
file.head(5)

,id,scene_id,person,gender,original_line,line,metadata,filename,processed_text,length
0,1,1,MONICA,F,Monica: There's nothing to tell! He's just som...,There's nothing to tell! He's just some guy I ...,There_EX 's_VBZ nothing_PN1 to_TO tell_VVI !_!...,0101.txt,there is nothing to tell ! he is just some guy...,11
1,101,1,JOEY,M,"Joey: C'mon, you're going out with the guy! Th...","C'mon, you're going out with the guy! There's ...",C'm_VV0 on_RP you_PPY 're_VBR going_VVG out_RP...,0101.txt,"come on , you are going out with the guy ! the...",14
2,201,1,CHANDLER,M,"Chandler: All right Joey, be nice. So does he...","Alright Joey, be nice. So does he have a hump?...",All_RR21 right_RR22 Joey_NP1 be_VBI nice_JJ ._...,0101.txt,"alright joey , be nice . so does he have a hum...",15
3,301,1,PHOEBE,F,"Phoebe: Wait, does he eat chalk?","Wait, does he eat chalk?",Wait_VV0 does_VDZ he_PPHS1 eat_VVI chalk_NN1 ?_?,0101.txt,"wait , does he eat chalk ?",5
4,401,1,PHOEBE,F,"Phoebe: Just, 'cause, I don't want her to go t...","Just, 'cause, I don't want her to go through w...",Just_RR 'cause_CS I_PPIS1 do_VD0 n't_XX want_V...,0101.txt,"just , because , i do not want her to go throu...",16


In [7]:
file.isnull().sum()

id                0
scene_id          0
person            0
gender            0
original_line     0
line              0
metadata          0
filename          0
processed_text    0
length            0
dtype: int64

In [8]:
temp_dataframe = file[['person','processed_text']]
remove_index = []
chat_pair = []
i = 0
while i <= (len(temp_dataframe)-2):
    raw_i = i
    
    while True:
#         print(raw_i,i, i+1)
        if temp_dataframe.iloc[i+1]['person'] == temp_dataframe.iloc[i]['person']:
            temp_dataframe.iloc[raw_i]['processed_text'] += ','+ temp_dataframe.iloc[i+1]['processed_text']
            chat_pair.append([temp_dataframe.iloc[raw_i]['processed_text'], temp_dataframe.iloc[raw_i+1]['processed_text']])
            remove_index.append(i+1)
        else:
            break
        i = i+1
        
    i += 1

In [9]:
temp_dataframe.drop(remove_index, inplace=True)

/home/sushil79g/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
temp_dataframe.head(5)

,person,processed_text
0,MONICA,there is nothing to tell ! he is just some guy...
1,JOEY,"come on , you are going out with the guy ! the..."
2,CHANDLER,"alright joey , be nice . so does he have a hum..."
3,PHOEBE,"wait , does he eat chalk ?,just , because , i ..."
5,MONICA,"okay , everybody relax . this is not even a da..."


In [11]:
# contractions.add("c'mon", 'come on')
list_of_word = list(temp_dataframe['processed_text'])

In [12]:
words = ' '.join(list_of_word).split()

In [13]:
from collections import Counter
wordcount = Counter(words)

In [14]:
vocab= set(words)

In [15]:
word_threshold = []
for word in vocab:
    if wordcount[word] >=3:
        word_threshold.append(word)
#     break

In [16]:
# word2ind ={{word, index} for word,index in enumerate(list(word_threshold))}

In [17]:
word2index = {}
index2word = {}
for index,word in enumerate(word_threshold):
    word2index.update({word:index})
    index2word.update({index:word})

In [18]:
from pprint import pprint
# pprint('total word in vocab',len(word_threshold))
len(word_threshold)

6756

In [ ]:
word2index

In [22]:
chat_pair[1]

['sounds like a date to me .,alright , so i am back in high school , i am standing in the middle of the cafeteria , and i realize i am totally naked .',
 'alright , so i am back in high school , i am standing in the middle of the cafeteria , and i realize i am totally naked .']

In [25]:
# batch_size = 5
# 'tied'
sentence = 'i am tied of working here. do you have any plans for today'

In [32]:
abc = [word2index[word] for word in sentence.split() if word in word2index.keys()]

In [ ]:
def find_max_length(sentences):
    whole_sentence = []
    for sentence in sentences:
        sentence_remove = []
        for word in sentence.split():
            if word in word2index.keys():
                sentence_remove.append(word)
        whole_sentence.append(sentence_remove)
    return whole_sentence, max([len(sent) for sent in whole_sentence])
print(find_max_length(list_of_word))

['there is nothing to tell ! he is just some guy i work with !',
 'come on , you are going out with the guy ! there is got to be something wrong with him !',
 'alright joey , be nice . so does he have a hump ? a hump and a hairpiece ?',
 'wait , does he eat chalk ?,just , because , i do not want her to go through what i went through with carl oh !',
 'okay , everybody relax . this is not even a date . it is just two people going out to dinner and not having sex .',
 'sounds like a date to me .,alright , so i am back in high school , i am standing in the middle of the cafeteria , and i realize i am totally naked .',
 'oh , yeah . had that dream .',
 'then i look down , and i realize there is a phone . . . there .',
 'instead of . . . ?',
 'that is right .',
 'never had that dream .',
 'no .',
 'all of a sudden , the phone starts to ring . now i do not know what to do , everybody starts looking at me .',
 'and they were not looking at you before ? !',
 'finally , i figure I would better 